In [ ]:
import pandas as pd
import numpy as np  
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
# Get the current directory
current_dir = os.getcwd()

# Navigate one folder up
parent_dir = os.path.dirname(current_dir)

# Where the files are located
data = "data/other"

# Navigate down into the "data" folder
data_dir = os.path.join(parent_dir, data)


In [ ]:

files = {
    "all_hotel_forecasts_df": "all_hotel_forecasts_From2021_to_Jan2025onwards_final_selected.csv",
    "all_hotel_historic_df": "all_hotel_historic_From2021_to_Jan2025_final_selected_final_selected.csv",
    "all_city_stars_historic_df": "all_city_stars_currency_historic_From2021_to_Jan2025_final_selected_final_selected.csv",
    "all_city_stars_forecasts_df": "city_stars_currency_forecasts_From2021_to_Jan2025onwards_final_selected.csv"
}

dataframes = {}

for var_name, file_name in files.items():
    try:
        file_path = os.path.join(data_dir, file_name)
        dataframes[var_name] = pd.read_csv(file_path)
        print(f"{var_name} loaded successfully.")
    except Exception as e:
        print(f"Error loading {var_name}: {e}")

In [ ]:
all_hotel_forecasts_df 
all_hotel_forecasts_df = all_hotel_forecasts_df[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'hotel', 'data_type']]
all_hotel_forecasts_df

In [ ]:
all_hotel_historic_df

In [ ]:
all_city_stars_historic_df 
all_city_stars_historic_df = all_city_stars_historic_df[['city_id', 'ds', 'city_stars_currency', 'y']]
all_city_stars_historic_df

In [ ]:
all_city_stars_forecasts_df  
all_city_stars_forecasts_df  = all_city_stars_forecasts_df [['ds', 'yhat_lower' , 'yhat_upper',  'yhat', 'city_stars_currency', 'data_type']]
all_city_stars_forecasts_df 

In [ ]:
data_lake_prd_314410_cz_hoteis = pd.read_csv('..\\data\\lookups\\data-lake-prd-314410.cz.hoteis.csv')
data_lake_prd_314410_cz_hoteis = data_lake_prd_314410_cz_hoteis[['Hotel_ID', 'Estrelas', 'Cidade_ID']]
data_lake_prd_314410_cz_hoteis

In [ ]:
#all_hotel_forecasts_df          -> join city and stars
#all_hotel_historic_df           -> join city and stars
#all_city_stars_historic_df      -> Extract  stars  from city_stars_currency
#all_city_stars_forecasts_df     -> Extract city and stars from city_stars_currency

In [ ]:
all_hotel_forecasts_df

In [ ]:
all_hotel_historic_df

In [ ]:
all_hotel_forecasts_df = pd.merge(all_hotel_forecasts_df, 
                     data_lake_prd_314410_cz_hoteis, 
                     left_on='hotel', 
                     right_on="Hotel_ID")

In [ ]:
all_hotel_historic_df = pd.merge(all_hotel_historic_df, 
                     data_lake_prd_314410_cz_hoteis, 
                     left_on='hotel_id', 
                     right_on="Hotel_ID")

In [ ]:
all_hotel_forecasts_df.drop(columns=['Hotel_ID'], inplace=True)
all_hotel_historic_df.drop(columns=['Hotel_ID'], inplace=True)

In [ ]:
all_city_stars_historic_df[['Cidade_ID', 'Estrelas', 'Currency']] = all_city_stars_historic_df['city_stars_currency'].str.split('__', expand=True)


In [ ]:
all_city_stars_forecasts_df[['Cidade_ID', 'Estrelas', 'Currency']] = all_city_stars_forecasts_df['city_stars_currency'].str.split('__', expand=True)
all_city_stars_forecasts_df

In [ ]:
#all_hotel_forecasts_df          
#all_hotel_historic_df           
#all_city_stars_historic_df      
#all_city_stars_forecasts_df     

In [ ]:
all_hotel_forecasts_df['Cidade_ID'] = all_hotel_forecasts_df['Cidade_ID'].astype(int)
all_hotel_historic_df['Cidade_ID'] = all_hotel_historic_df['Cidade_ID'].astype(int)
all_city_stars_historic_df['Cidade_ID'] = all_city_stars_historic_df['Cidade_ID'].astype(int)
all_city_stars_forecasts_df['Cidade_ID'] = all_city_stars_forecasts_df['Cidade_ID'].astype(int)

In [ ]:
#all_hotel_forecasts_df          
#all_hotel_historic_df           
#all_city_stars_historic_df      
#all_city_stars_forecasts_df  

In [ ]:
all_hotel_forecasts_df.dropna(inplace=True)
all_hotel_historic_df.dropna(inplace=True)
all_city_stars_historic_df.dropna(inplace=True)
all_city_stars_forecasts_df.dropna(inplace=True)


In [ ]:
all_hotel_forecasts_df['Estrelas'] = all_hotel_forecasts_df['Estrelas'].astype(int)
all_hotel_historic_df['Estrelas'] = all_hotel_historic_df['Estrelas'].astype(int)
all_city_stars_historic_df['Estrelas'] = all_city_stars_historic_df['Estrelas'].astype(int)
all_city_stars_forecasts_df['Estrelas'] = all_city_stars_forecasts_df['Estrelas'].astype(int)

In [ ]:
all_hotel_historic_df = all_hotel_historic_df.merge(
    all_city_stars_historic_df[['ds', 'Cidade_ID', 'Estrelas', 'y']],
    on=['ds', 'Cidade_ID', 'Estrelas'],
    how='inner'
)


In [ ]:
all_hotel_forecasts_df = all_hotel_forecasts_df.merge(
    all_city_stars_forecasts_df[['ds', 'Cidade_ID', 'Estrelas', 'data_type',  'yhat' , 'yhat_lower'  , 'yhat_upper']],
    on=['ds', 'Cidade_ID', 'Estrelas',  'data_type'],
    how='inner'
)

In [ ]:
all_hotel_forecasts_df.columns

In [ ]:
to_plot = all_hotel_forecasts_df[all_hotel_forecasts_df['hotel'] == 13252] #759 # 4397

In [ ]:
to_plot['ds'] = pd.to_datetime(to_plot['ds']).dt.tz_localize(None)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# Set the seaborn theme
sns.set_theme()

# Plot observed data and predictions with different colors
plt.figure(figsize=(16, 8))
plt.plot(to_plot['ds'], to_plot['yhat_x'], label="Predicted Data Hotel", color="#27445D", alpha=0.8)
plt.plot(to_plot['ds'], to_plot['yhat_y'], label="Predicted Data City", color="firebrick", alpha=0.8)


# Add confidence intervals for predictions
plt.fill_between(
    to_plot['ds'], to_plot['yhat_lower_x'], to_plot['yhat_upper_x'], color="#27445D", alpha=0.15, label="Uncertainty Interval Hotel"
)

plt.fill_between(
    to_plot['ds'], to_plot['yhat_lower_y'], to_plot['yhat_upper_y'], color="firebrick", alpha=0.15, label="Uncertainty Interval City"
)

# Customize x-axis for better readability
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))  # Set ticks every 3 months
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # Format as Year-Month
plt.gcf().autofmt_xdate(rotation=90)  # Rotate x-axis labels vertically

# Customize the plot
plt.title(f"Hotel {to_plot['hotel'].unique()} Data Forecast & Respective City {to_plot['Cidade_ID'].unique()} Forecast", fontsize=16)
plt.xlabel("\n Exact Date", fontsize=14)
plt.ylabel("Average Daily Price (Local Currency) \n", fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, linestyle="--", alpha=0.6)


# Save the plot before showing
#plt.savefig(f"../out/figures/city_stars_currency_forecast_{combined_df_hist['city_stars_currency'].unique()}.png", dpi=300, bbox_inches="tight")

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
all_hotel_forecasts_df.to_csv("../out/all_hotel_with_city_forecasts_df_From2021_to_Jan2025.csv", index=False)

In [ ]:
unique_hotels = all_hotel_forecasts_df['hotel'].unique().tolist()

In [ ]:
# Set the seaborn theme
sns.set_theme()

# Get unique hotel IDs
unique_hotels

# Create a single PDF to store all plots
with PdfPages("../out/figures/pdf/single_hotel_all_hotel_forecasts_df_365_2025.pdf") as pdf:
    for hotel in unique_hotels:
        # Filter data for the current hotel
        single_hotel_all_hotel_forecasts_df = all_hotel_forecasts_df[all_hotel_forecasts_df["hotel"] == hotel]
        
        single_hotel_all_hotel_forecasts_df["ds"] = pd.to_datetime(single_hotel_all_hotel_forecasts_df["ds"], errors="coerce")

        # Create a new figure
        plt.figure(figsize=(16, 8))


        # Plot hotel data
        plt.plot(single_hotel_all_hotel_forecasts_df["ds"], single_hotel_all_hotel_forecasts_df["yhat_x"], label="Observed Data Hotel", color="#E195AB", alpha=0.8)
    
        # Plot city data
        plt.plot(single_hotel_all_hotel_forecasts_df["ds"], single_hotel_all_hotel_forecasts_df["yhat_y"], label="Predicted Data City", color="#27445D", alpha=0.6)
    

        # Add confidence intervals for predictions
        plt.fill_between(
            single_hotel_all_hotel_forecasts_df["ds"], 
            single_hotel_all_hotel_forecasts_df["yhat_lower_x"], 
            single_hotel_all_hotel_forecasts_df["yhat_upper_x"], 
            color="#E195AB", alpha=0.2, label="Uncertainty Interval Hotel"
        )

        # Add confidence intervals for predictions
        plt.fill_between(
            single_hotel_all_hotel_forecasts_df["ds"], 
            single_hotel_all_hotel_forecasts_df["yhat_lower_y"], 
            single_hotel_all_hotel_forecasts_df["yhat_upper_y"], 
            color="#27445D", alpha=0.2, label="Uncertainty Interval City"
        )

        # Customize x-axis
        plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))  
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  
        plt.gcf().autofmt_xdate(rotation=90)  

        # Customize the plot
        plt.title(f"Hotel {single_hotel_all_hotel_forecasts_df['hotel'].unique()} Data Forecast & Respective City {single_hotel_all_hotel_forecasts_df['Cidade_ID'].unique()} Forecast", fontsize=16)
        
        plt.xlabel("\n Exact Date", fontsize=14)
        plt.ylabel("Average Daily Price (Local Currency) \n", fontsize=14)
        plt.legend(fontsize=12)
        plt.grid(True, linestyle="--", alpha=0.6)

        # Save the current figure to the PDF
        pdf.savefig()
        plt.close()  # Close the figure to free memory

print("All plots saved successfully in 'single_hotel_all_hotel_forecasts_df_365_2025.pdf'!")

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Get unique hotel IDs
unique_hotels

output_dir = "../out/figures/html/"


for hotel in unique_hotels:
    # Extract data for the current hotel
    single_hotel_all_hotel_forecasts_df = all_hotel_forecasts_df[all_hotel_forecasts_df["hotel"] == hotel]
    
    single_hotel_all_hotel_forecasts_df["ds"] = pd.to_datetime(single_hotel_all_hotel_forecasts_df["ds"], errors="coerce")

    # Create interactive plot
    fig = go.Figure()

    # Add observed data
    fig.add_trace(go.Scatter(
        x=single_hotel_all_hotel_forecasts_df["ds"],
        y=single_hotel_all_hotel_forecasts_df["yhat_x"],
        mode="lines",
        name="Predicted Data Hotel",
        line=dict(color="#E195AB")
    ))

    # Add predicted data
    fig.add_trace(go.Scatter(
        x=single_hotel_all_hotel_forecasts_df["ds"],
        y=single_hotel_all_hotel_forecasts_df["yhat_y"],
        mode="lines",
        name="Predicted Data City",
        line=dict(color="#27445D", dash="dash")
    ))

    # Add confidence interval
    fig.add_trace(go.Scatter(
        x=single_hotel_all_hotel_forecasts_df["ds"].tolist() + single_hotel_all_hotel_forecasts_df["ds"].tolist()[::-1],
        y=single_hotel_all_hotel_forecasts_df["yhat_upper_x"].tolist() + single_hotel_all_hotel_forecasts_df["yhat_lower_x"].tolist()[::-1],
        fill="toself",
        fillcolor="rgba(178, 34, 34, 0.3)",  # 30% opacity
        line=dict(color="rgba(178, 34, 34, 0.0)"),  # Full opacity
        name="Uncertainty Interval Hotel"
    ))

   # Add confidence interval
    fig.add_trace(go.Scatter(
        x=single_hotel_all_hotel_forecasts_df["ds"].tolist() + single_hotel_all_hotel_forecasts_df["ds"].tolist()[::-1],
        y=single_hotel_all_hotel_forecasts_df["yhat_upper_y"].tolist() + single_hotel_all_hotel_forecasts_df["yhat_lower_y"].tolist()[::-1],
        fill="toself",
        fillcolor="rgba(39, 68, 93, 0.3)",
        line=dict(color="rgba(39, 68, 93, 0)") ,
        name="Uncertainty Interval City"
    ))


    # Customize layout
    fig.update_layout(
        title=f"Hotel {single_hotel_all_hotel_forecasts_df['hotel'].unique()} Data Forecast & Respective City {single_hotel_all_hotel_forecasts_df['Cidade_ID'].unique()} Forecast",
        xaxis_title="Exact Date",
        yaxis_title="Average Daily Price (Local Currency)",
        xaxis=dict(tickformat="%Y-%m", tickangle=90),
        template="plotly_white"
    )

    # Save as HTML file
    output_path = os.path.join(output_dir, f"single_hotel_all_hotel_forecasts_df_{hotel}.html")
    fig.write_html(output_path)

print("All interactive plots saved successfully!")


In [ ]:
# Set the seaborn theme
sns.set_theme()

# Get unique hotel IDs
unique_hotels

# Loop through each city_currency_ids
for hotel in unique_hotels:
    # Filter data for the current city_stars_currency_ids
    single_hotel_all_hotel_forecasts_df = all_hotel_forecasts_df[all_hotel_forecasts_df["hotel"] == hotel]

    single_hotel_all_hotel_forecasts_df["ds"] = pd.to_datetime(single_hotel_all_hotel_forecasts_df["ds"], errors="coerce")

    # Create a new figure
    plt.figure(figsize=(16, 8))
    
    # Plot hotel data
    plt.plot(single_hotel_all_hotel_forecasts_df["ds"], single_hotel_all_hotel_forecasts_df["yhat_x"], label="Observed Data Hotel", color="#E195AB", alpha=0.8)
    
    # Plot city data
    plt.plot(single_hotel_all_hotel_forecasts_df["ds"], single_hotel_all_hotel_forecasts_df["yhat_y"], label="Predicted Data City", color="#27445D", alpha=0.6)
    
    # Add confidence intervals for predictions
    plt.fill_between(
        single_hotel_all_hotel_forecasts_df["ds"], 
        single_hotel_all_hotel_forecasts_df["yhat_lower_x"], 
        single_hotel_all_hotel_forecasts_df["yhat_upper_x"], 
        color="#E195AB", alpha=0.2, label="Uncertainty Interval Hotel"
    )

    # Add confidence intervals for predictions
    plt.fill_between(
        single_hotel_all_hotel_forecasts_df["ds"], 
        single_hotel_all_hotel_forecasts_df["yhat_lower_y"], 
        single_hotel_all_hotel_forecasts_df["yhat_upper_y"], 
        color="#27445D", alpha=0.2, label="Uncertainty Interval City"
    )

    # Customize x-axis
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))  
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  
    plt.gcf().autofmt_xdate(rotation=90)  

    # Customize the plot
    plt.title(f"Hotel {single_hotel_all_hotel_forecasts_df['hotel'].unique()} Data Forecast & Respective City {single_hotel_all_hotel_forecasts_df['Cidade_ID'].unique()} Forecast", fontsize=16)
    plt.xlabel("\n Exact Date", fontsize=14)
    plt.ylabel("Average Daily Price (Local Currency) \n", fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, linestyle="--", alpha=0.6)

    # Save the plot with hotel_id in the filename
    plt.savefig(f"../out/figures/png/single_hotel_all_hotel_forecasts_df_{hotel}.png", dpi=300, bbox_inches="tight")
    
    # Close the plot to free memory before the next iteration
    plt.close()

print("All plots saved successfully!")